In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
print("Hello world")

Hello world


In [106]:
url = "http://ufcstats.com/event-details/7956f026e2672c47"

In [107]:
r = requests.get(url)

In [108]:
html_content = r.content


In [109]:
soup = BeautifulSoup(r.content, 'html.parser')


In [110]:
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
    Stats | UFC
  </title>
<meta content="" name="description"/>
<meta content="" name="viewport"/>
<link href="/blocks/main.css?ver=187297" rel="stylesheet"/>
<script src="/js/vendor/modernizr-2.6.2.min.js"></script>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'auto');
    ga('send

In [111]:
location = None
info_box = soup.find("div", class_="b-list__info-box")
if info_box:
    
    items = info_box.find_all("li", class_="b-list__box-list-item")
    for item in items:
     title = item.find("i", class_="b-list__box-item-title")
    if title and "Location" in title.text:
        location = item.get_text(strip=True).replace("Location:", "").strip()
location

'Abu Dhabi, Abu Dhabi, United Arab Emirates'

In [112]:
table = soup.find_all("table",class_='b-fight-details__table')[0]

In [113]:
table

<table class="b-fight-details__table b-fight-details__table_style_margin-top b-fight-details__table_type_event-details js-fight-table" style="display: table;">
<thead class="b-fight-details__table-head">
<tr class="b-fight-details__table-row">
<th class="b-fight-details__table-col">
    W/L
  </th>
<th class="b-fight-details__table-col l-page_align_left">
    Fighter
  </th>
<th class="b-fight-details__table-col">
    Kd
  </th>
<th class="b-fight-details__table-col">
    Str
  </th>
<th class="b-fight-details__table-col">
    Td
  </th>
<th class="b-fight-details__table-col">
    Sub
  </th>
<th class="b-fight-details__table-col l-page_align_left">
    Weight class
  </th>
<th class="b-fight-details__table-col l-page_align_left">
    Method
  </th>
<th class="b-fight-details__table-col">
    Round
  </th>
<th class="b-fight-details__table-col">
    Time
  </th>
</tr>
</thead>
<tbody class="b-fight-details__table-body">
<tr class="b-fight-details__table-row b-fight-details__table-row__

In [8]:
headers = table.find_all("th")

In [115]:
headers

['W/L',
 'Fighter',
 'Kd',
 'Str',
 'Td',
 'Sub',
 'Weight class',
 'Method',
 'Round',
 'Time']

In [116]:
headers = [i.text  for i in headers ]
headers

AttributeError: 'str' object has no attribute 'text'

In [11]:
headers = [i.strip(' ').strip('\n').strip(' ')  for i in headers ]
headers

['W/L',
 'Fighter',
 'Kd',
 'Str',
 'Td',
 'Sub',
 'Weight class',
 'Method',
 'Round',
 'Time']

In [12]:
df = pd.DataFrame(columns=headers)
df

,W/L,Fighter,Kd,Str,Td,Sub,Weight class,Method,Round,Time


In [117]:
data = soup.find_all('td')
data

[<td class="b-fight-details__table-col b-fight-details__table-col_style_align-top">
 <p class="b-fight-details__table-text">
 <a class="b-flag b-flag_style_bordered" href="http://ufcstats.com/fight-details/6390c8e74630473e"><i class="b-flag__inner"><i class="b-flag__text">nc<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
 </p>
 <p class="b-fight-details__table-text">
 <a class="b-flag b-flag_style_bordered" href="http://ufcstats.com/fight-details/6390c8e74630473e"><i class="b-flag__inner"><i class="b-flag__text">nc<i class="b-flag__corner"></i><i class="b-flag__corner-substrate"></i></i></i></a>
 </p>
 </td>,
 <td class="b-fight-details__table-col l-page_align_left" style="width:100px">
 <p class="b-fight-details__table-text">
 <a class="b-link b-link_style_black" href="http://ufcstats.com/fighter-details/399afbabc02376b5">
               Tom Aspinall
             </a>
 </p>
 <p class="b-fight-details__table-text">
 <a class="b-link b-link_style_black

In [7]:
import re
import pandas as pd

In [8]:
def ufc_scraper(link):

    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Extract location once per event
    location = None
    info_box = soup.find("div", class_="b-list__info-box")
    if info_box:
        items = info_box.find_all("li", class_="b-list__box-list-item")
        for item in items:
            title = item.find("i", class_="b-list__box-item-title")
            if title and "Location" in title.text:
                location = item.get_text(strip=True).replace("Location:", "").strip()

    rows = soup.select("tbody tr")
    all_fights = []

    def clean_text(txt):
        return re.sub(r"\s+", " ", txt).strip()

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 10:
            continue

        fighters = [p.text.strip() for p in cols[1].select("p")]
        kd = [clean_text(p.text) for p in cols[2].select("p")]
        sig_str = [clean_text(p.text) for p in cols[3].select("p")]
        td = [clean_text(p.text) for p in cols[4].select("p")]
        sub = [clean_text(p.text) for p in cols[5].select("p")]
        res = None
        winner = None

        if clean_text(cols[7].text) == 'CNC':
            res = 'nc'
            winner = "-"
            
            

        fight = {
            "fighter_1": fighters[0],
            "fighter_2": fighters[1],
            "fighter_1_kd": int(kd[0]) if kd[0].isdigit() else 0,
            "fighter_2_kd": int(kd[1]) if kd[1].isdigit() else 0,
            "fighter_1_sig_str": int(sig_str[0]),
            "fighter_2_sig_str": int(sig_str[1]),
            "fighter_1_td": int(td[0]),
            "fighter_2_td": int(td[1]),
            "fighter_1_sub": int(sub[0]),
            "fighter_2_sub": int(sub[1]),
            "result": res or cols[0].text.strip(),
            "winner": winner or (fighters[0] if cols[0].text.strip().lower() == "win" else fighters[1]),
            "weight_class": clean_text(cols[6].text),
            "method": clean_text(cols[7].text),
            "rounds": int(cols[8].text.strip()),
            "time": cols[9].text.strip(),
            "location": location
        }

        all_fights.append(fight)

    # Convert list of dicts to DataFrame
    df = pd.DataFrame(all_fights)
    return df

    

In [9]:
df_original = pd.DataFrame()

In [10]:

def full_Scraper(last_page):

    df_original = pd.DataFrame()

    for i in range(1 ,int(last_page)+1):
        
        r = requests.get("http://ufcstats.com/statistics/events/completed?page="+str(i))
        soup = BeautifulSoup(r.content, 'html.parser')

        events=soup.find_all("a", class_="b-link")
        event_links = []
        for a in events:  # `events` is your list of <a> tags
            link = a.get("href")
            if link:
                event_links.append(link)

        if i == 1:
            event_links = event_links[1:]

        for url in event_links:
            
            df1 = ufc_scraper(url)
            df_original = pd.concat([df_original,df1], ignore_index=True)

    return df_original
            
            
                
        
    

In [11]:
or_df_frame = full_Scraper(29)

In [12]:
or_df_frame

,fighter_1,fighter_2,fighter_1_kd,fighter_2_kd,fighter_1_sig_str,fighter_2_sig_str,fighter_1_td,fighter_2_td,fighter_1_sub,fighter_2_sub,result,winner,weight_class,method,rounds,time,location
0,Islam Makhachev,Jack Della Maddalena,0,0,30,18,4,0,0,0,win,Islam Makhachev,Welterweight,U-DEC,5,5:00,"New York City, New York, USA"
1,Valentina Shevchenko,Zhang Weili,0,0,41,32,5,0,0,0,win,Valentina Shevchenko,Women's Flyweight,U-DEC,5,5:00,"New York City, New York, USA"
2,Michael Morales,Sean Brady,1,0,24,8,0,0,0,0,win,Michael Morales,Welterweight,KO/TKO Punches,1,3:27,"New York City, New York, USA"
3,Carlos Prates,Leon Edwards,1,0,21,12,0,2,0,0,win,Carlos Prates,Welterweight,KO/TKO Punch,2,1:28,"New York City, New York, USA"
4,Benoit Saint Denis,Beneil Dariush,1,0,3,3,0,0,0,0,win,Benoit Saint Denis,Lightweight,KO/TKO Punch,1,0:16,"New York City, New York, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8196,Josh Barnett,Gan McGee,0,0,17,10,1,2,1,0,win,Josh Barnett,Super Heavyweight,KO/TKO Elbows,2,4:34,"Atlantic City, New Jersey, USA"
8197,Andrei Arlovski,Aaron Brink,0,0,2,0,1,0,2,0,win,Andrei Arlovski,Heavyweight,SUB Armbar,1,0:55,"Atlantic City, New Jersey, USA"
8198,Jens Pulver,John Lewis,1,0,1,0,0,0,0,0,win,Jens Pulver,Lightweight,KO/TKO Punch,1,0:15,"Atlantic City, New Jersey, USA"
8199,Mark Hughes,Alex Stiebling,0,0,9,4,2,0,3,1,win,Mark Hughes,Middleweight,U-DEC,2,5:00,"Atlantic City, New Jersey, USA"


In [13]:
or_df_frame.to_csv("./dataset/ufc_data.csv",index=False)